# ML Model Horse Racing Predictions
#### Create an ML model that can predict whether a horse has a good chance of winning a race, based on the horse's features

## Preprocessing

In [17]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
cleaned_data_df = pd.read_csv("Data_Racing/International_Data/cleaned_data_df.csv")
cleaned_data_df.head()

,Unnamed: 0,ages,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,father,weight,res_win
0,1,4yo+,3218.0,10,6.0,7.0,0.444444,1,96.0,89.0,110.0,Getaway,65,0.0
1,26,5yo+,3218.0,5,8.0,1.0,0.266667,1,115.0,33.0,109.0,Beneficial,73,0.0
2,27,5yo+,3218.0,5,9.0,2.0,0.125000,0,108.0,29.0,103.0,Beneficial,68,0.0
3,43,4yo+,4827.0,6,7.0,13.0,0.076923,0,95.0,39.0,83.0,Kalanisi,63,1.0
4,44,4yo+,4827.0,6,8.0,2.0,0.076923,0,119.0,64.0,108.0,Ask,71,0.0


In [18]:
# Determine the number of unique values in each column.
cleaned_data_df.nunique()

Unnamed: 0      24922
ages               16
metric             52
ncond              13
age                14
saddle             29
decimalPrice       79
isFav               2
RPR               165
TR                157
OR                149
father            327
weight             32
res_win             2
dtype: int64

## Binning (not used for now)

In [19]:
# Look at value counts for binning
res_win_counts = cleaned_data_df["res_win"].value_counts()
res_win_counts

0.0    21753
1.0     3169
Name: res_win, dtype: int64

In [22]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
res_win_to_replace = list(res_win_counts[winning_time_counts < 900].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [23]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()
classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [24]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts.loc[classification_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [25]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 800].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

## Post Binning

In [20]:
# Convert categorical data to numeric with `pd.get_dummies`
converted_data = pd.get_dummies(cleaned_data_df)
converted_data.head()

,Unnamed: 0,metric,ncond,age,saddle,decimalPrice,isFav,RPR,TR,OR,...,father_Well Chosen,father_Westerner,father_Whipper,father_Winged Love,father_Wootton Bassett,father_Yeats,father_Youmzain,father_Zanzibari,father_Zebedee,father_Zoffany
0,1,3218.0,10,6.0,7.0,0.444444,1,96.0,89.0,110.0,...,0,0,0,0,0,0,0,0,0,0
1,26,3218.0,5,8.0,1.0,0.266667,1,115.0,33.0,109.0,...,0,0,0,0,0,0,0,0,0,0
2,27,3218.0,5,9.0,2.0,0.125000,0,108.0,29.0,103.0,...,0,0,0,0,0,0,0,0,0,0
3,43,4827.0,6,7.0,13.0,0.076923,0,95.0,39.0,83.0,...,0,0,0,0,0,0,0,0,0,0
4,44,4827.0,6,8.0,2.0,0.076923,0,119.0,64.0,108.0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# Split our preprocessed data into our features and target arrays
y = converted_data['res_win']
X = converted_data.drop(columns=['res_win'])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [24]:
input_features = len(X_train_scaled[0])
input_features

354

## Compile, Train and Evaluate the Model

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
neurons_hidden_layer1 = 256
neurons_hidden_layer2 = 148
neurons_hidden_layer3 = 76

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer1, activation="relu", input_dim = input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_hidden_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               90880     
                                                                 
 dense_1 (Dense)             (None, 148)               38036     
                                                                 
 dense_2 (Dense)             (None, 76)                11324     
                                                                 
 dense_3 (Dense)             (None, 1)                 77        
                                                                 
Total params: 140,317
Trainable params: 140,317
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [27]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 250)

Epoch 1/250
585/585 [==============================] - 1s 975us/step - loss: 0.3450 - accuracy: 0.8706
Epoch 2/250
585/585 [==============================] - 1s 925us/step - loss: 0.2973 - accuracy: 0.8760
Epoch 3/250
585/585 [==============================] - 1s 1ms/step - loss: 0.2725 - accuracy: 0.8829
Epoch 4/250
585/585 [==============================] - 1s 919us/step - loss: 0.2518 - accuracy: 0.8907
Epoch 5/250
585/585 [==============================] - 1s 933us/step - loss: 0.2341 - accuracy: 0.8966
Epoch 6/250
585/585 [==============================] - 1s 972us/step - loss: 0.2198 - accuracy: 0.9004
Epoch 7/250
585/585 [==============================] - 1s 997us/step - loss: 0.2089 - accuracy: 0.9055
Epoch 8/250
585/585 [==============================] - 1s 956us/step - loss: 0.1977 - accuracy: 0.9112
Epoch 9/250
585/585 [==============================] - 1s 917us/step - loss: 0.1854 - accuracy: 0.9147
Epoch 10/250
585/585 [==============================] - 1s 920us/step - los

585/585 [==============================] - 1s 916us/step - loss: 0.0251 - accuracy: 0.9923
Epoch 80/250
585/585 [==============================] - 1s 931us/step - loss: 0.0175 - accuracy: 0.9933
Epoch 81/250
585/585 [==============================] - 1s 919us/step - loss: 0.0186 - accuracy: 0.9939
Epoch 82/250
585/585 [==============================] - 1s 917us/step - loss: 0.0194 - accuracy: 0.9943
Epoch 83/250
585/585 [==============================] - 1s 918us/step - loss: 0.0133 - accuracy: 0.9961
Epoch 84/250
585/585 [==============================] - 1s 916us/step - loss: 0.0139 - accuracy: 0.9958
Epoch 85/250
585/585 [==============================] - 1s 916us/step - loss: 0.0168 - accuracy: 0.9951
Epoch 86/250
585/585 [==============================] - 1s 917us/step - loss: 0.0213 - accuracy: 0.9932
Epoch 87/250
585/585 [==============================] - 1s 919us/step - loss: 0.0160 - accuracy: 0.9942
Epoch 88/250
585/585 [==============================] - 1s 990us/step - loss:

585/585 [==============================] - 1s 948us/step - loss: 0.0102 - accuracy: 0.9974
Epoch 158/250
585/585 [==============================] - 1s 998us/step - loss: 0.0114 - accuracy: 0.9969
Epoch 159/250
585/585 [==============================] - 1s 952us/step - loss: 0.0090 - accuracy: 0.9971
Epoch 160/250
585/585 [==============================] - 1s 954us/step - loss: 0.0058 - accuracy: 0.9982
Epoch 161/250
585/585 [==============================] - 1s 950us/step - loss: 0.0098 - accuracy: 0.9975
Epoch 162/250
585/585 [==============================] - 1s 960us/step - loss: 0.0041 - accuracy: 0.9986
Epoch 163/250
585/585 [==============================] - 1s 944us/step - loss: 0.0046 - accuracy: 0.9983
Epoch 164/250
585/585 [==============================] - 1s 958us/step - loss: 0.0075 - accuracy: 0.9974
Epoch 165/250
585/585 [==============================] - 1s 948us/step - loss: 0.0096 - accuracy: 0.9971
Epoch 166/250
585/585 [==============================] - 1s 953us/ste

585/585 [==============================] - 1s 928us/step - loss: 0.0074 - accuracy: 0.9983
Epoch 236/250
585/585 [==============================] - 1s 934us/step - loss: 0.0034 - accuracy: 0.9988
Epoch 237/250
585/585 [==============================] - 1s 922us/step - loss: 0.0079 - accuracy: 0.9981
Epoch 238/250
585/585 [==============================] - 1s 915us/step - loss: 0.0055 - accuracy: 0.9987
Epoch 239/250
585/585 [==============================] - 1s 917us/step - loss: 0.0033 - accuracy: 0.9989
Epoch 240/250
585/585 [==============================] - 1s 948us/step - loss: 0.0047 - accuracy: 0.9982
Epoch 241/250
585/585 [==============================] - 1s 920us/step - loss: 0.0039 - accuracy: 0.9991
Epoch 242/250
585/585 [==============================] - 1s 970us/step - loss: 0.0018 - accuracy: 0.9992
Epoch 243/250
585/585 [==============================] - 1s 962us/step - loss: 0.0021 - accuracy: 0.9990
Epoch 244/250
585/585 [==============================] - 1s 928us/ste

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

195/195 - 0s - loss: 1.2729 - accuracy: 0.8777 - 337ms/epoch - 2ms/step
Loss: 1.2728846073150635, Accuracy: 0.8777082562446594


In [33]:
# Export our model to HDF5 file
# nn.save('h5_files/AlphabetSoup_Model4.h5')